<a href="https://colab.research.google.com/github/charlaburnett/change_detection_ukraine/blob/main/mariupol_osint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
#Geotag News Lookup

In [84]:
#Add location Geocoding support
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="sar_change_osint")

for article in news_data:
    title = article["title"]
    pub_date = article["publishedAt"]
    location_guess = "Mariupol"  # Simplified assumption

    try:
        loc = geolocator.geocode(location_guess)
        coords = (loc.latitude, loc.longitude) if loc else ("N/A", "N/A")
    except:
        coords = ("N/A", "N/A")

    print(f"{title}")
    print(f"{pub_date}")
    print(f"{coords}")
    print(f"{article['url']}\n")


Ukraine war briefing: Soldiers captured by Russia in Mariupol among those released in prisoner swap
2025-06-27T00:58:13Z
(47.0957648, 37.5499621)
https://www.theguardian.com/world/2025/jun/27/ukraine-war-briefing-soldiers-captured-by-russia-in-mariupol-among-those-released-in-prisoner-swap



Ukraine war briefing: Deaths in Kharkiv as Russia continues strikes on civilian targets
2025-06-11T00:03:50Z
(47.0957648, 37.5499621)
https://www.theguardian.com/world/2025/jun/11/ukraine-war-briefing-deaths-in-kharkiv-as-russia-continues-strikes-on-civilian-targets



Russia hands prison terms to Ukrainian 'Azov' fighters
2025-03-26T00:00:00Z
(47.0957648, 37.5499621)
https://www.rte.ie/news/ukraine/2025/0326/1504223-azov-fighters/

Actors who sheltered in Ukraine's Mariupol theatre bring story of bombing to the stage
2025-01-14T01:38:00Z
(47.0957648, 37.5499621)
https://news.sky.com/story/actors-who-sheltered-in-ukraines-mariupol-theatre-bring-story-of-bombing-to-the-stage-13288135



‘If you hear the siren, go to the shelter’: the wrenching play about the bombing of a theatre in Ukraine
2025-01-10T15:16:12Z
(47.0957648, 37.5499621)
https://www.theguardian.com/stage/2025/jan/10/if-you-hear-the-siren-go-to-the-shelter-the-wrenching-play-about-the-bombing-of-a-theatre-in-ukraine

‘Mariupolis 2' is a grim portrait of war. Lithuanian diplomat wishes it was never made
2024-11-02T03:33:54Z
(47.0957648, 37.5499621)
https://theprint.in/feature/around-town/mariupolis-2-is-a-grim-portrait-of-war-lithuania-ukraine-russia/2338019/



Russia accused of using hunger as a weapon of war in Ukraine
2024-06-13T16:39:56Z
(47.0957648, 37.5499621)
https://edition.cnn.com/2024/06/13/europe/russia-ukraine-mariupol-hunger-war-crime-intl/index.html

Ukraine war briefing: Russian strike on city of Kryvyi Rih kills nine, including five children
2024-06-13T00:29:00Z
(47.0957648, 37.5499621)
https://www.theguardian.com/world/article/2024/jun/13/ukraine-war-briefing-russian-strike-on-city-of-kryvyi-rih-kills-nine-including-five-children

Russia accused of ‘deliberate’ starvation tactics in Mariupol in submission to ICC
2024-06-12T23:01:10Z
(47.0957648, 37.5499621)
https://www.theguardian.com/world/article/2024/jun/13/russia-accused-of-deliberate-starvation-tactics-in-mariupol-in-submission-to-icc



Ukraine war briefing: Washington clears Azov brigade for US weapons and training
2024-06-12T01:18:12Z
(47.0957648, 37.5499621)
https://www.theguardian.com/world/article/2024/jun/12/ukraine-war-briefing-washington-clears-azov-brigade-for-us-weapons-and-training



In [85]:
import import_ipynb
import sys

sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [86]:
prompt = """You are an OSINT analyst reviewing reported events in Mariupol between March and June 2022.
The following healines may relate to damage, airstrikes, and conflict events.
Your taks is to:
1. Identify key incidents (who/what/when/where),
2. Group them by date or location if appropriate,
3. Indicate confidence level if details are unclear,
4. Suggest whether any might explain observed SAR-detected structural changes.
Headlines:

"""

for article in news_data:
  title = article.get("title", "")
  date = article.get("publishedAt", "") [:10]
  prompt += f"- ({date}) {title}\n"

In [87]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
# openai_key.py should contain:
# import openai
# openai.api_key = "sk-..."

# gnews_key.py should contain:
# GNEWS_API_KEY = "your-gnews-api-key"

from openai_key import openai
from gnews_key import GNEWS_API_KEY
from openai import OpenAI



In [89]:
import requests
from geopy.geocoders import Nominatim

client = OpenAI(api_key=openai.api_key)
geolocator = Nominatim(user_agent="sar_change_osint")

query = "Mariupol airstrike OR bombing OR war"
start_date = "2022-03-01"
end_date = "2022-06-30"
url = f"https://gnews.io/api/v4/search?q={query}&from={start_date}&to={end_date}&lang=en&max=10&token={GNEWS_API_KEY}"

response = requests.get(url)
news_data = response.json().get("articles", [])
print(f"Retrieved {len(news_data)} articles")


Retrieved 10 articles


In [90]:
prompt = """
You are an OSINT analyst. Below are news headlines about Mariupol, Ukraine from March to June 2022.

Your task:
1. Extract reported events: what happened, where, when.
2. Group events by time/location/theme.
3. Flag any that likely caused structural damage (airstrikes, demolitions, sieges).
4. Estimate confidence based on repetition or specificity.

Headlines:
"""

for article in news_data:
    title = article.get("title", "")
    date = article.get("publishedAt", "")[:10]
    prompt += f"- ({date}) {title}\n"


In [91]:
gpt_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.4
)

summary = gpt_response.choices[0].message.content
print("GPT-Generated OSINT Summary:\n")
print(summary)


🧠 GPT-Generated OSINT Summary:

### Extracted Events

1. **Prisoner Swap and Captures:**
   - **Event:** Ukrainian soldiers captured by Russia in Mariupol were released in a prisoner swap.
   - **Location:** Mariupol, Ukraine
   - **Date:** June 27, 2025
   - **Confidence:** High (specific mention of soldiers captured in Mariupol and released)

2. **Theatre Bombing:**
   - **Event:** Bombing of a theatre in Mariupol, Ukraine.
   - **Location:** Mariupol, Ukraine
   - **Date:** Not specified in the headlines, but implied through retrospective accounts and plays.
   - **Confidence:** High (repeated references to the event in different contexts)

3. **Prison Terms for Azov Fighters:**
   - **Event:** Russia handed prison terms to Ukrainian 'Azov' fighters.
   - **Location:** Not specifically mentioned, but related to Mariupol due to Azov's defense of the city.
   - **Date:** March 26, 2025
   - **Confidence:** Medium (specific event but location inferred)

4. **Starvation Tactics:**
   - 